# Data Processes
state_dict = {"sleep":0, "preseizure": 1}

In [1]:
import numpy as np
import os 
import pickle
import sys
from dtw import dtw
from tqdm.notebook import tqdm
sys.path.append("../")
from util.util_file import time_add
import pandas as pd
print("Start")

Start


## 1.构建滤波数据和原始数据的映射 

In [2]:
# filter
filter_pre_1_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/filter/pre_1"
filter_pre_2_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/filter/pre_2"
filter_sleep_path_dir = "../visualization_feature/raw_data_time_sequentially/sleep/BDP/filter"

# non-filter
non_filter_pre_1_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/no_filter/pre_1"
non_filter_pre_2_path_dir = "../visualization_feature/raw_data_time_sequentially/preseizure/BDP/no_filter/pre_2"
non_filter_sleep_path_dir = "../visualization_feature/raw_data_time_sequentially/sleep/BDP/no_filter"

# channel list 
hot_spot_pre_1_path = "./data/pre_1/BDP_SZ1_pre_seizure_raw.txt"
hot_spot_pre_2_path = "./data/pre_2/BDP_SZ1_pre_seizure_raw.txt"
hot_spot_sleep_path = "./data/sleep/BDP_Sleep_raw.txt"

生成滤波和非滤波切片的映射, 需要提前运行，滤波数据和非滤波数据的路径进行映射

In [3]:
fi_pre_1_names = os.listdir(filter_pre_1_path_dir)
fi_pre_2_names = os.listdir(filter_pre_2_path_dir)
fi_sleep_names = os.listdir(filter_sleep_path_dir)
# sorted by time 
fi_pre_1_names.sort()
fi_pre_2_names.sort()
fi_sleep_names.sort()
print("fi_pre_1:{}, fi_pre_2:{}, fi_sleep:{}".format(len(fi_pre_1_names), len(fi_pre_2_names), len(fi_sleep_names)))

nfi_pre_1_names = os.listdir(non_filter_pre_1_path_dir)
nfi_pre_2_names = os.listdir(non_filter_pre_2_path_dir)
nfi_sleep_names = os.listdir(non_filter_sleep_path_dir)
nfi_pre_1_names.sort()
nfi_pre_2_names.sort()
nfi_sleep_names.sort()
print("nfi_pre_1:{}, nfi_pre_2:{}, nfi_sleep:{}".format(len(nfi_pre_1_names), len(nfi_pre_2_names), len(nfi_sleep_names)))


fi_nfi_dict = {}
label = 1
fi_names = [fi_pre_1_names, fi_pre_2_names, fi_sleep_names]
nfi_names = [nfi_pre_1_names, nfi_pre_2_names, nfi_sleep_names]
nfi_dir = [non_filter_pre_1_path_dir, non_filter_pre_2_path_dir, non_filter_sleep_path_dir]
for _index, fi_ida in enumerate(fi_names):
    _nfi_d = nfi_dir[_index]
    nfi_ida = nfi_names[_index]
    label = 1 if _index < 2 else 0
    for fi_n, nfi_n in zip(fi_ida, nfi_ida):
        nfi_ab_path = os.path.join(_nfi_d, nfi_n)
        fi_nfi_dict[fi_n] = {"nfi_path":nfi_ab_path, "label": label}
        
print(len(fi_nfi_dict))

fi_pre_1:309, fi_pre_2:1783, fi_sleep:7199
nfi_pre_1:309, nfi_pre_2:1783, nfi_sleep:7199
9291


将映射表进行保存

保存格式：{fi_id:{"nfi_id":nfi_id, "label":0/1 }}

fi_id: 经过滤波切片的id，  nfi_if: 没有经过滤波的切片id



In [4]:
sava_file = "./data/fi_nfi_id.pkl"
with open(sava_file, 'wb') as f:
    pickle.dump(fi_nfi_dict, f)
    print("映射表保存成功！")

映射表保存成功！


## 2.特征重新映射,需要将特征和id序列号进行映射

In [5]:
feature_pre_1_path = "./data/pre_1/BDP-feature.npy"
feature_pre_2_path = "./data/pre_2/BDP-feature.npy"
feature_sleep_path = "./data/sleep/BDP-feature.npy"

feature_pre_1_data = np.load(feature_pre_1_path)
feature_pre_2_data = np.load(feature_pre_2_path)
feature_sleep_data = np.load(feature_sleep_path)

print(feature_pre_1_data.shape, feature_pre_2_data.shape, feature_sleep_data.shape)
feature_id_data_dict = {}
concatenate_feature = np.concatenate((feature_pre_1_data, feature_pre_2_data, feature_sleep_data), axis=0)
print(concatenate_feature.shape)
for id, data in zip(fi_pre_1_names+fi_pre_2_names+fi_sleep_names, concatenate_feature):
    feature_id_data_dict[id] = data
print(len(feature_id_data_dict))

(309, 50) (1783, 50) (7199, 50)
(9291, 50)
9291


In [6]:
save_path = "./data/feature_id_data_dict.pkl"
with open(save_path, 'wb') as f:
    pickle.dump(feature_id_data_dict, f)
    print("特征id序列化完成！")

特征id序列化完成！


## 3.选出模型判定正确的特征

In [7]:
path_val_prediction = "./data/BDP_val_prediction.pkl"
val_prediction_dict = np.load(path_val_prediction, allow_pickle=True)
print(len(val_prediction_dict))

9074


In [8]:
feature_true_id_prediction = {}
count = 0
for id, data in feature_id_data_dict.items():
    if id in val_prediction_dict.keys():
        if val_prediction_dict[id]["ground truth"] == val_prediction_dict[id]["prediction"]:
            feature_true_id_prediction[id] = data
    else:
        count += 1

print(len(feature_true_id_prediction))
print("不存在的id数：{}".format(count))

# 选择特征的写回
save_path = "./data/feature_true_id_prediction.pkl"
with open(save_path, 'wb') as f:
    pickle.dump(feature_true_id_prediction, f)
    print("模型写入成功！")
    

4224
不存在的id数：217
模型写入成功！


## 4. 每个特征和时间的映射关系

In [9]:
feature_id_time_dict = {}
pre_1_time = (23, 1, 52)
file_name_1 = "pre_seizure_1"
pre_2_time = (3, 1, 52)
file_name_2 = "pre_seizure_2"
sleep_time = (23, 1, 52)
file_name_3 = "normal_sleep"
_time = pre_1_time
Relative_Time = (0, 0, 0)
for id in fi_pre_1_names:
    feature_id_time_dict[id] = {"Absolute time":_time, "Relative time": Relative_Time, "File name":file_name_1}
    Relative_Time = time_add(Relative_Time[0], Relative_Time[1], Relative_Time[2], 2)
    _time = time_add(_time[0], _time[1], _time[2], 2)

_time = pre_2_time
Relative_Time = (0, 0, 0)
for id in fi_pre_2_names:
    feature_id_time_dict[id] = {"Absolute time":_time, "Relative time": Relative_Time, "File name":file_name_2}
    Relative_Time = time_add(Relative_Time[0], Relative_Time[1], Relative_Time[2], 2)
    _time = time_add(_time[0], _time[1], _time[2], 2)

_time = sleep_time
Relative_Time = (0, 0, 0)
for id in fi_sleep_names:
    feature_id_time_dict[id] = {"Absolute time":_time, "Relative time": Relative_Time, "File name":file_name_3}
    Relative_Time = time_add(Relative_Time[0], Relative_Time[1], Relative_Time[2], 2)
    _time = time_add(_time[0], _time[1], _time[2], 2)

save_path = "./data/feature_id_time_dict.pkl"
print(len(feature_id_time_dict))
with open(save_path, 'wb') as f:
    pickle.dump(feature_id_time_dict, f)
    print("特征时间映射完成！")
    

9291
特征时间映射完成！


## 5. 将特征和特征选取的信道进行映射{feature_id:channel_No}

In [10]:
hot_spot_pre_1_list = pd.read_csv(hot_spot_pre_1_path, sep=',')
hot_spot_pre_2_list = pd.read_csv(hot_spot_pre_2_path, sep=',')
hot_spot_sleep_list = pd.read_csv(hot_spot_sleep_path, sep=',')
feature_id_time_spatial= {}
run_list = [hot_spot_pre_1_list, hot_spot_pre_2_list, hot_spot_sleep_list]
run_list_dict = [fi_pre_1_names, fi_pre_2_names, fi_sleep_names]
for _index, r in tqdm(enumerate(run_list)):
    _time = r["time_location"]
    _spatial = r["spatial_location"] 
    fi_names = run_list_dict[_index]
    for i, id_name  in enumerate(fi_names):
        _time_line = _time[i].split("-")[0]
        _spatial_line = _spatial[i].split("-")[0]
        record_line = {"time":int(_time_line), "spatial":int(_spatial_line)}
        feature_id_time_spatial[id_name] = record_line

with open("./data/feature_id_time_spatial.pkl", 'wb') as f:
    pickle.dump(feature_id_time_spatial, f)
    print("id 热点信息映射完成！")


id 热点信息映射完成！


## 6.计算特征之间的distance

In [11]:
read_path = "./data/feature_true_id_prediction.pkl"
with open(read_path, 'rb') as f:
    feature_true_id_prediction = pickle.load(f)
    print("{} 读取成功！, 数据长度:{}".format(read_path, len(feature_true_id_prediction)))
#print(len(feature_true_id_prediction))
def similarity_DTW(s1, s2):
    ratio = 1 # 设定的放缩系数，避免数据的相似度过于集中
    euclidean_norm = lambda x, y: np.abs(ratio * (x - y))
    d, cost_matrix, acc_cost_matrix, path = dtw(s1, s2, dist=euclidean_norm)
    # score = 1 - np.tanh(d)  # 相似度的评分【0,1】 0： 完全不同， 1： 完全相同
    return d
score_similarity_DTW = np.zeros((len(feature_true_id_prediction), len(feature_true_id_prediction)))

./data/feature_true_id_prediction.pkl 读取成功！, 数据长度:4224


In [12]:
id_list = list(feature_true_id_prediction.keys())
for i, id in enumerate(tqdm(id_list)):
    s1 = feature_true_id_prediction[id]
    for j in range(0, i+1):
        id_2 = id_list[j]
        s2 = feature_true_id_prediction[id_2]
        score = similarity_DTW(s1, s2)
        score_similarity_DTW[i][j] = score
        score_similarity_DTW[j][i] = score
print(score_similarity_DTW)
save_DTW_data = "./processing/DTW_similarity_matrix.npy"
np.save(save_DTW_data, score_similarity_DTW)


[[0.         0.01438541 0.02431704 ... 0.02458381 0.02068662 0.02572145]
 [0.01438541 0.         0.01598483 ... 0.00420379 0.00311675 0.00466537]
 [0.02431704 0.01598483 0.         ... 0.01647419 0.01543484 0.01856046]
 ...
 [0.02458381 0.00420379 0.01647419 ... 0.         0.00185057 0.00180244]
 [0.02068662 0.00311675 0.01543484 ... 0.00185057 0.         0.002706  ]
 [0.02572145 0.00466537 0.01856046 ... 0.00180244 0.002706   0.        ]]


### 7. 可选模块的运行分析-（主要是针对于优化过后的cluster在进行一聚类分析）

In [ ]:
# cluster:8 ID:1
Best_label_cluster_path = "./processing/Best_label_cluster.pkl"
DTW_distance_matrix_path = "./processing/DTW_distance_matrix.npy"
Best_label_cluster = np.load(Best_label_cluster_path, allow_pickle=True)
print(len(Best_label_cluster))

In [ ]:
DTW_distance_matrix = np.load(DTW_distance_matrix_path)
Cluster_8_ID_1 = Best_label_cluster[1]
print(len(Cluster_8_ID_1))
print(DTW_distance_matrix.shape)
feature_true_id_prediction_path = "./data/feature_true_id_prediction.pkl"
feature_true_id_prediction = np.load(feature_true_id_prediction_path, allow_pickle=True)
fi_name_list = list(feature_true_id_prediction.keys())
print(len(fi_name_list))

In [ ]:
Cluster_8_ID_distance = np.zeros((len(Cluster_8_ID_1), len(Cluster_8_ID_1)))
for i, f_id_a in tqdm(enumerate(Cluster_8_ID_1)):
    for j in range(i+1):
        f_id_b = Cluster_8_ID_1[j]
        index_a = fi_name_list.index(f_id_a)
        index_b = fi_name_list.index(f_id_b)
        score = DTW_distance_matrix[index_a, index_b]
        Cluster_8_ID_distance[i][j] = score
        Cluster_8_ID_distance[j][i] = score
Cluster_8_ID_1_result = {"fi_id_list":Cluster_8_ID_1,
                         "DTW_distance_matrix":Cluster_8_ID_distance
                         }
save_path = "./processing/Cluster_8_ID_1__fid_list_distance.pkl"
with open(save_path, 'wb') as f:
    pickle.dump(Cluster_8_ID_1_result, f)
    print("文件写入成功！cluster_8_id_1_result")

save_path = "./processing/feature_true_id_prediction_cluster_8_1.pkl"
feature_true_id_prediction_cluster_8_1 = {}
for f_id in Cluster_8_ID_1:
    feature_true_id_prediction_cluster_8_1[f_id] = feature_true_id_prediction[f_id]
with open(save_path, 'wb') as f:
    pickle.dump(feature_true_id_prediction_cluster_8_1, f)
    print("特定feature 写入成功！")
    

In [ ]:
print(score_similarity_DTW.shape)


